In [2]:
%logstop
%logstart -rtq ~/.logs/nlp.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the result of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [5]:
import gzip
import ujson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

In [6]:
data[0]

{'reviewerID': 'A238V1XTSK9NFE',
 'asin': 'B00004VX3T',
 'reviewerName': 'Andrew Lynn',
 'helpful': [2, 2],
 'reviewText': "I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automatical

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [7]:
ratings = [x['overall'] for x in data]

In [8]:
ratings[:10]

[5.0, 1.0, 4.0, 5.0, 3.0, 5.0, 3.0, 5.0, 4.0, 5.0]

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

In [10]:
class KeySelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [row[self.key] for row in X]

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from spacy.lang.en import STOP_WORDS
import numpy as np

In [12]:
STOP_WORDS

{'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'front',
 'full',
 'further',
 'get',
 'give',
 'g

In [13]:
len(STOP_WORDS)

305

In [14]:
bag_of_words_model = Pipeline([
    ('get_text', KeySelector('reviewText')),
    ('vectorizer', CountVectorizer(stop_words=STOP_WORDS)),
    ('ridge', GridSearchCV(Ridge(), param_grid={'alpha': np.logspace(1.5,2.5,3)}, 
                           cv=5, n_jobs=2, verbose=1))
])

In [15]:
bag_of_words_model.fit(data, ratings)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:  2.3min finished


Pipeline(memory=None,
         steps=[('get_text', KeySelector(key='reviewText')),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words={'a', 'about', 'above', 'across',
                                             'after',...
                 GridSearchCV(cv=5, error_score='raise-deprecating',
                              estimator=Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                                              random_sta

In [18]:
bag_of_words_model.named_steps['ridge'].best_params_

{'alpha': 316.22776601683796}

In [19]:
bag_of_words_model.named_steps['ridge'].best_estimator_

Ridge(alpha=316.22776601683796, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [23]:
bag_of_words_model.score(data, ratings)  #  R^2 score

0.39866990623047294

In [24]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

Your score:  1.1067615266155737


In [25]:
bag_of_words_model.named_steps['ridge'].best_params_

{'alpha': 316.22776601683796}

## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalized to your model to improve performance.

In [11]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
normalized_model = Pipeline([
    ('get_text', KeySelector('reviewText')),
    ('vectorizer', TfidfVectorizer(stop_words=STOP_WORDS, max_features=2500)),
    ('ridge', GridSearchCV(Ridge(), param_grid={'alpha': np.logspace(-2,1,5)}, 
                          cv=5, n_jobs=2, verbose=1))
])

In [20]:
normalized_model.fit(data, ratings)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:   44.0s finished


Pipeline(memory=None,
         steps=[('get_text', KeySelector(key='reviewText')),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=2500,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'a', 'abou...
                 GridSearchCV(cv=5, error_score='raise-deprecating',
                              estimator=Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                                              random_state=None, solv

In [21]:
normalized_model.score(data, ratings)

0.44262501888527217

In [22]:
normalized_model.named_steps['ridge'].best_params_

{'alpha': 1.7782794100389228}

In [23]:
grader.score.nlp__normalized_model(normalized_model.predict)

Your score:  0.9066943335899673


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [ ]:
bigrams_model = Pipeline([
    ('get_text', KeySelector('reviewText')),
    ('vectorizer', TfidfVectorizer(stop_words=STOP_WORDS, ngram_range=(2,2))),
    ('ridge', GridSearchCV(Ridge(), param_grid={'alpha': np.logspace(-2,1,5)}, 
                           cv=5, verbose=1))
])

bigrams_model.fit(data, ratings)

In [14]:
bigrams_model.score(data, ratings)

0.912382865986365

In [15]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score:  0.684050080011421


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the review is negative given the word "terrible" appears in the text. Using these probabilities, we can derive a polarity score for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [20]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

--2020-10-29 12:48:08--  http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.217.67.116
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.217.67.116|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2970853 (2.8M) [application/x-gzip]
Saving to: ‘./data/amazon_one_and_five_star_reviews.json.gz’

     0K .......... .......... .......... .......... ..........  1% 34.9M 0s
    50K .......... .......... .......... .......... ..........  3% 54.0M 0s
   100K .......... .......... .......... .......... ..........  5% 38.8M 0s
   150K .......... .......... .......... .......... ..........  6% 33.6M 0s
   200K .......... .......... .......... .......... ..........  8% 38.3M 0s
   250K .......... .......... .......... .......... .......... 10% 50.9M 0s
   300K .......... .......... .......... .......... .......

To avoid memory issue, we can delete the older data.

In [21]:
del data, ratings

In [22]:
del bag_of_words_model

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = [x['overall'] for x in data_polarity]


In [27]:
pipe = Pipeline([
    ('get_text', KeySelector('reviewText')),
    ('vectorizer', TfidfVectorizer(stop_words=STOP_WORDS)),
    ('estimator', MultinomialNB())
])

pipe.fit(data_polarity, ratings)

Pipeline(memory=None,
         steps=[('get_text', KeySelector(key='reviewText')),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'a', 'abou...
                                             'also', 'although', 'always', 'am',
                                             'among', 'amongst', 'amount', 'an',
                                             'and', 'another', 'any', 'anyhow',
                                             'anyone', 'anything', 'anyway',
                                 

In [28]:
nb = pipe.named_steps['estimator']


In [29]:
nb.feature_log_prob_

array([[ -8.64595801,  -9.6369024 , -10.86439331, ..., -10.19829036,
        -10.71077731, -10.62308687],
       [ -9.35207212,  -9.30288304, -10.57273656, ..., -10.81783738,
        -10.81783738, -10.81783738]])

In [33]:
log_probs = nb.feature_log_prob_

In [32]:
index_to_token = pipe.named_steps['vectorizer'].get_feature_names()
index_to_token[2000:2010]

['acoustic',
 'acoustical',
 'acoustically',
 'acoustics',
 'acoustimass',
 'acquainted',
 'acquire',
 'acquired',
 'acquiring',
 'acquisition']

In [34]:
log_probs.shape

(2, 25422)

In [36]:
polarity = log_probs[0,:] - log_probs[1,:]

In [37]:
polarity

array([ 0.7061141 , -0.33401936, -0.29165675, ...,  0.61954703,
        0.10706008,  0.19475051])

In [39]:
index = np.argsort(polarity)

In [40]:
index

array([11288,  3610, 17646, ..., 19021, 24531, 18497])

In [42]:
extreme_index = np.hstack((index[:25], index[:-26:-1]))

In [43]:
extreme_index

array([11288,  3610, 17646, 16559, 14790, 17123,  2511, 21610, 13921,
       11980,  8880,  4036, 16917,   524,  5010, 10947,  3303, 17124,
        7961,  6376, 16718, 22331,  4353,  9267, 18541, 18497, 24531,
       19021, 19024, 25052, 12809, 22407, 19023, 10275, 23902, 25054,
       25048, 23073,  6878,  3776, 17082, 23418,  3304, 11459, 23709,
       21425, 18051,  7418, 22636, 18509])

In [47]:
top_50 = [index_to_token[i] for i in extreme_index]

In [48]:
top_50

['highly',
 'beat',
 'protects',
 'perfect',
 'monopod',
 'portrait',
 'amazing',
 'sturdy',
 'macro',
 'incredible',
 'excellent',
 'bokeh',
 'pleased',
 '200mm',
 'charm',
 'handy',
 'awesome',
 'portraits',
 'dslr',
 'crisp',
 'photography',
 'telephoto',
 'buck',
 'fantastic',
 'regrets',
 'refund',
 'waste',
 'return',
 'returning',
 'worst',
 'junk',
 'terrible',
 'returned',
 'garbage',
 'useless',
 'worthless',
 'worse',
 'trash',
 'defective',
 'beware',
 'poor',
 'unacceptable',
 'awful',
 'horrible',
 'unreliable',
 'stopped',
 'randomly',
 'disappointing',
 'threw',
 'refused']

In [49]:
grader.score.nlp__most_polar(top_50)

Your score:  1.0


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [ ]:
from sklearn.decomposition import NMF
 

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*